<a href="https://colab.research.google.com/github/ameympatil/MIT-ML/blob/main/Keras_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
import pandas as pd
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential
from keras.layers import Activation, Dense

In [41]:
df = pd.read_csv('/content/Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [42]:
df.shape

(10000, 14)

In [43]:
ohe = OneHotEncoder(sparse = False)
df['Geography'] = ohe.fit_transform(df[['Geography']])
df['Gender'] = ohe.fit_transform(df[['Gender']])

In [44]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,1.0,1.0,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,0.0,1.0,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,1.0,1.0,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,1.0,1.0,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,0.0,1.0,43,2,125510.82,1,1,1,79084.10,0


In [45]:
X = df.iloc[:,3:13]
y = df['Exited']

In [46]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=0)

In [47]:
X_train

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
7681,641,1.0,0.0,33,2,146193.60,2,1,1,55796.83
9031,541,1.0,0.0,39,7,0.00,2,1,0,19823.02
3691,590,1.0,1.0,76,5,160979.68,1,0,1,13848.58
202,516,0.0,0.0,50,5,0.00,1,0,1,146145.93
5625,508,1.0,1.0,60,7,143262.04,1,1,1,129562.74
...,...,...,...,...,...,...,...,...,...,...
9225,594,0.0,1.0,32,4,120074.97,2,1,1,162961.79
4859,794,0.0,1.0,22,4,114440.24,1,1,1,107753.07
3264,738,1.0,0.0,35,5,161274.05,2,1,0,181429.87
9845,590,0.0,1.0,38,9,0.00,2,1,1,148750.16


In [48]:
def create_model(layers,activation):
  model = Sequential()
  for i, nodes in enumerate(layers):
    if i==0:
      model.add(Dense(nodes,input_dim=X_train.shape[1]))
      model.add(Activation(activation))
    else:
      model.add(Dense(nodes))
      model.add(Activation(activatin))
  model.add(Dense(1))
  model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
  return model
model = KerasClassifier(build_fn=create_model, verbose=0)

In [49]:
layers = [[20],[40,20],[45,30,15]]
activations = ['sigmoid','relu']
param_grid = dict(layers=layers,activation=activations,batch_size = [128,256], epochs = [30])
grid = GridSearchCV(estimator=model,param_grid=param_grid)

In [50]:
grid_result = grid.fit(X_train,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
40 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
40 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/keras/wrappers/scikit_learn.py", line 219, in fit
    return super(KerasClassifier, self).fit(x, y, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/keras/wrappers/scikit_learn.py", line 153, in fit
    self.model = self.build_fn(**self.filter_sk_params(self.buil

In [51]:
[grid_result.best_score_,grid_result.best_params_]

[0.6988571286201477,
 {'activation': 'sigmoid', 'batch_size': 128, 'epochs': 30, 'layers': [20]}]

In [52]:
pred_y = grid.predict(X_test)

In [53]:
y_pred = [pred_y > 0.5]

In [54]:
y_pred

[array([[False],
        [False],
        [False],
        ...,
        [False],
        [False],
        [False]])]